<a href="https://colab.research.google.com/github/cuauhtemocbe/Python-Basics/blob/main/Buffers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creador de Buffers

In [1]:
# Instalación de librerías
!pip install geopandas
!pip install folium

     |████████████████████████████████| 994 kB 8.3 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 
     |████████████████████████████████| 6.3 MB 40.1 MB/s 


In [2]:
import os 

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon

import folium
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

In [4]:
FILE_NAME = "/content/drive/MyDrive/Colab Notebooks/Spatial Analyst/data/multiple.csv" # [EDITABLE] Cambia por el nombre del archivo
# SISTEMA DE COORDENADAS
# UTM 14: 32614
# UTM 15: 32615
# UTM 16: 32616
CRS = 32616 #  [EDITABLE] Cambiar por el crs de tu preferencia
df = pd.read_csv(FILE_NAME) 
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.X, df.Y), crs=CRS)

## Información de tu archivo

In [5]:
# Para ver los primeros elementos en tu csv
gdf.head()

,ID,no_sitio,X,Y,tipo_veg,perturbacion,geometry
0,1,1,305823,2298208,NaN,NaN,POINT (305823.000 2298208.000)
1,2,2,305807,2298309,NaN,NaN,POINT (305807.000 2298309.000)
2,3,3,305844,2298433,NaN,NaN,POINT (305844.000 2298433.000)
3,4,48,371146,2294158,veg secundaria SBC,NaN,POINT (371146.000 2294158.000)
4,5,51,371092,2294247,veg secundaria SMC,NaN,POINT (371092.000 2294247.000)


# Creación del buffer

In [6]:
buffer_size = 10 #  [EDITABLE], valor por defaul 10 metros
gdf['geometry'] = gdf.buffer(buffer_size)

# Guardar tu shape

In [7]:
NAME = "buffers"  #  [EDITABLE], cambia por el nombre que quieres
os.makedirs('buffer')
gdf.to_file('tmp/' + NAME + ".shp")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


DriverIOError: ignored

# Mapa

In [8]:
gdf.head()

,ID,no_sitio,X,Y,tipo_veg,perturbacion,geometry
0,1,1,305823,2298208,NaN,NaN,"POLYGON ((305833.000 2298208.000, 305832.952 2..."
1,2,2,305807,2298309,NaN,NaN,"POLYGON ((305817.000 2298309.000, 305816.952 2..."
2,3,3,305844,2298433,NaN,NaN,"POLYGON ((305854.000 2298433.000, 305853.952 2..."
3,4,48,371146,2294158,veg secundaria SBC,NaN,"POLYGON ((371156.000 2294158.000, 371155.952 2..."
4,5,51,371092,2294247,veg secundaria SMC,NaN,"POLYGON ((371102.000 2294247.000, 371101.952 2..."


In [11]:
POINT_ID = 0
try:
  point = gdf.loc[[POINT_ID]].to_crs(4326)['geometry'][POINT_ID].centroid
except:
  print(f'Valor máximo permitido es {gdf.shape[0]}')

In [12]:
# Niveles de zoom
# 1: Todo el mundo
# 4: México
# 8: Estado
# 12: Municipio
ZOOM = 8  #  [EDITABLE]
layer_gj = gdf.to_crs(epsg='4326').to_json()
mapa = folium.Map(location=[point.y, point.x],
                  zoom_start=ZOOM,
                  tiles='cartodbpositron')

points = folium.features.GeoJson(layer_gj)
mapa.add_child(points)
mapa